In [174]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict

In [175]:
songs = pd.read_csv('songdata (1).csv')

In [176]:
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [177]:
songs.isna().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [186]:
songs['text'] = songs['text'].replace(r'\n','')


In [191]:
str=''
for i in range(0,len(songs)):
    str=str+songs['artist'][i]+" "

In [192]:
artists=str.split()

In [193]:
unique_artists=set(artists)

In [195]:
len(unique_artists)

989

In [179]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_num = len(recom_song)
        print(f'The {rec_num} recommended songs for {song} are:')
        for i in range(rec_num):
            print(f"\nNumber {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]}") 
        
        
    def recommend(self, recommendation):
        song = recommendation['song']
        number_songs = recommendation['number_songs']
        recom_song = self.matrix_similar[song][:number_songs]
        self._print_message(song=song, recom_song=recom_song)

In [ ]:
# class CollaborativeFilteringRecommender:

In [180]:
def get_selection(artist,song,text,rec_num):
    global songs
    x={'artist':artist,'song':song,'text':text}
    x=pd.DataFrame([x])
    songs = songs.sample(n=3000).drop('link', axis=1).reset_index(drop=True)
    songs=pd.concat([songs,x])
    tfidf = TfidfVectorizer(analyzer='word', stop_words='english')
    lyrics_matrix = tfidf.fit_transform(songs['text'])
    cosine_similarities = cosine_similarity(lyrics_matrix) 
    similarities = {}
    for i in range(len(cosine_similarities)):
        similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
        similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'].iloc[x], songs['artist'].iloc[x]) for x in similar_indices][1:]
    recommedations = ContentBasedRecommender(similarities)
    recommendation = {
    "song": songs['song'].iloc[3000],
    "number_songs": rec_num }
    rec_songs=recommedations.recommend(recommendation)
    return rec_songs